In [112]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [304]:
trips = pd.read_csv('../detroit/output/trips.csv', index_col=0)

/home/ubuntu/anaconda3/envs/baus/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


#### Last trip of the day is always a home trip

In [271]:
trips.groupby('person_id')['purpose'].last().value_counts()

Home    3821771
Name: purpose, dtype: int64

#### All persons trips are sequentially ordered

In [278]:
res = trips.groupby('person_id').apply(lambda x: all(x.trip_id.values[:-1] < x.trip_id.values[1:]))

In [280]:
res.value_counts()

True    3821771
dtype: int64

In [12]:
jtp = pd.read_csv('../detroit/output/joint_tour_participants.csv', index_col=0)

In [13]:
tours = pd.read_csv('../detroit/output/tours.csv', index_col=0)

/home/ubuntu/anaconda3/envs/baus/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (17,19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [306]:
tours.head()

,tour_id,person_id,tour_type,tour_type_count,tour_type_num,tour_num,tour_count,tour_category,number_of_participants,destination,...,start,end,duration,tdd,composition,tour_mode,atwork_subtour_frequency,parent_tour_id,stop_frequency,primary_purpose
0,31,0,school,1,1,1,1,mandatory,1,1.0,...,7.0,15.0,8.0,45.0,NaN,DRIVE_LOC,NaN,NaN,0out_0in,school
1,72,1,school,1,1,1,1,mandatory,1,314.0,...,8.0,15.0,7.0,61.0,NaN,SHARED3FREE,NaN,NaN,1out_0in,school
2,195,4,school,1,1,1,1,mandatory,1,486.0,...,9.0,14.0,5.0,75.0,NaN,SHARED3FREE,NaN,NaN,0out_0in,school
3,236,5,school,1,1,1,1,mandatory,1,167.0,...,8.0,19.0,11.0,65.0,NaN,SHARED3PAY,NaN,NaN,0out_1in,school
4,7132,173,work,1,1,1,1,mandatory,1,245.0,...,8.0,21.0,13.0,67.0,NaN,DRIVE_LOC,no_subtours,NaN,0out_0in,work


In [14]:
plans = pd.read_csv('../detroit/output/plans.csv', index_col=0)

In [65]:
persons = pd.read_csv('../detroit/output/persons.csv', index_col=0)

/home/ubuntu/anaconda3/envs/baus/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [66]:
zones = pd.read_csv('../detroit/data/land_use.csv')

In [20]:
tours.columns

Index(['tour_id', 'person_id', 'tour_type', 'tour_type_count', 'tour_type_num',
       'tour_num', 'tour_count', 'tour_category', 'number_of_participants',
       'destination', 'origin', 'household_id', 'start', 'end', 'duration',
       'tdd', 'composition', 'tour_mode', 'atwork_subtour_frequency',
       'parent_tour_id', 'stop_frequency', 'primary_purpose'],
      dtype='object')

In [49]:
tours['tour_type'].value_counts()

work        1544835
school       955794
shopping     689959
othdiscr     545536
escort       539157
othmaint     403647
eat          309141
eatout       245081
social       195154
business      69892
maint         49112
Name: tour_type, dtype: int64

In [19]:
tours[tours['tour_category'] == 'joint'].shape[0] == len(jtp['tour_id'].unique())

True

In [28]:
tours[tours['tour_category'] == 'joint']['tour_mode'].value_counts()

SHARED2FREE    37842
SHARED2PAY     35587
SHARED3FREE    17906
SHARED3PAY     16860
WALK_LOC        1250
WALK             597
BIKE             291
DRIVE_LOC        210
Name: tour_mode, dtype: int64

#### joint tours never have atwork subtours

In [29]:
tours[tours['tour_category'] == 'joint']['atwork_subtour_frequency'].value_counts()

Series([], Name: atwork_subtour_frequency, dtype: int64)

In [30]:
tours[tours['tour_category'] == 'joint']['parent_tour_id'].value_counts()

Series([], Name: parent_tour_id, dtype: int64)

#### joint tours whose members have atwork subtours?

In [37]:
atwork_tours = tours[(tours['tour_category'] == 'atwork')]

In [38]:
atwork_tours

,tour_id,person_id,tour_type,tour_type_count,tour_type_num,tour_num,tour_count,tour_category,number_of_participants,destination,...,start,end,duration,tdd,composition,tour_mode,atwork_subtour_frequency,parent_tour_id,stop_frequency,primary_purpose
214256,11525,281,eat,1,1,1,1,atwork,1,909.0,...,10.0,10.0,0.0,85.0,NaN,WALK,NaN,11560.0,0out_0in,atwork
214257,17142,418,eat,1,1,1,1,atwork,1,242.0,...,10.0,10.0,0.0,85.0,NaN,SHARED3PAY,NaN,17177.0,0out_0in,atwork
214258,23534,574,business,1,1,1,1,atwork,1,410.0,...,12.0,14.0,2.0,114.0,NaN,SHARED2PAY,NaN,23573.0,0out_2in,atwork
214259,23579,575,eat,1,1,1,1,atwork,1,277.0,...,11.0,13.0,2.0,101.0,NaN,WALK_LOC,NaN,23614.0,1out_0in,atwork
214260,27821,678,maint,1,1,1,1,atwork,1,535.0,...,15.0,15.0,0.0,145.0,NaN,DRIVEALONEPAY,NaN,27837.0,0out_0in,atwork
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5547303,180060975,4391731,eat,1,1,1,1,atwork,1,983.0,...,8.0,9.0,1.0,55.0,NaN,DRIVEALONEPAY,NaN,180061010.0,0out_0in,atwork
5547304,180090760,4392457,maint,1,1,1,1,atwork,1,993.0,...,10.0,13.0,3.0,88.0,NaN,SHARED2FREE,NaN,180090776.0,0out_0in,atwork
5547305,180113004,4393000,eat,1,1,1,1,atwork,1,62.0,...,13.0,14.0,1.0,125.0,NaN,SHARED3PAY,NaN,180113039.0,0out_1in,atwork
5547306,180119892,4393168,eat,1,1,1,1,atwork,1,138.0,...,13.0,14.0,1.0,125.0,NaN,SHARED3PAY,NaN,180119927.0,0out_0in,atwork


In [46]:
passengers_w_atwork_tours = jtp[(jtp['person_id'].isin(atwork_tours['person_id'])) & (jtp['participant_num'] > 1)]

#### types of joint tours whose passengers have their own atwork tours

In [51]:
tours[tours['tour_id'].isin(passengers_w_atwork_tours['tour_id'])]['tour_category'].value_counts()

joint    8530
Name: tour_category, dtype: int64

In [48]:
tours[tours['tour_id'].isin(passengers_w_atwork_tours['tour_id'])]['tour_type'].value_counts()

othdiscr    3498
eatout      1423
shopping    1391
othmaint    1352
social       866
Name: tour_type, dtype: int64

#### joint tours are never mandatory tours!?!

In [52]:
tours[tours['tour_category'] == 'joint']['tour_type'].value_counts()

othdiscr    35022
shopping    24667
othmaint    23753
eatout      13968
social      13133
Name: tour_type, dtype: int64

#### what do passenger's daily plans look like

In [55]:
plans[plans['person_id'].isin(passengers_w_atwork_tours.sample()['person_id'])]

,trip_id,person_id,PlanElementIndex,ActivityElement,ActivityType,x,y,departure_time
4424858,NaN,596338.0,1,activity,Home,-82.885191,42.585014,6.014
4424859,195599177.0,596338.0,2,leg,NaN,NaN,NaN,NaN
4424860,NaN,596338.0,3,activity,Home,-82.909918,42.579792,13.907
4424861,195598897.0,596338.0,4,leg,NaN,NaN,NaN,NaN
4424862,NaN,596338.0,5,activity,Home,-82.910130,42.440059,16.223
4424863,195598901.0,596338.0,6,leg,NaN,NaN,NaN,NaN
4424864,NaN,596338.0,7,activity,Home,-82.914809,42.420992,18.604
4424865,195599181.0,596338.0,8,leg,NaN,NaN,NaN,NaN
4424866,NaN,596338.0,9,activity,Home,-82.885191,42.585014,NaN


#### Why are all activity types = "home"???

In [64]:
plans['ActivityType'].value_counts()

Home    17965766
Name: ActivityType, dtype: int64

In [57]:
trips[trips['person_id'] == 596338].sort_values('depart')

,trip_id,person_id,household_id,tour_id,primary_purpose,trip_num,outbound,trip_count,purpose,destination,origin,depart,trip_mode
10692281,195599177,596338,229795,24449897,work,1,True,1,work,156,947,6.0,WALK_LOC
10692279,195598897,596338,229795,24449862,atwork,1,True,1,atwork,114,156,13.0,WALK
10692280,195598901,596338,229795,24449862,atwork,1,False,1,Work,156,114,16.0,WALK
10692282,195599181,596338,229795,24449897,work,1,False,1,Home,947,156,18.0,WALK_LOC


In [63]:
trips[trips['tour_id'].isin(tours[tours['tour_id'].isin(jtp[jtp['person_id'] == 596338]['tour_id'])]['tour_id'])]

,trip_id,person_id,household_id,tour_id,primary_purpose,trip_num,outbound,trip_count,purpose,destination,origin,depart,trip_mode
10692277,195598313,596335,229795,24449789,eatout,1,True,1,eatout,212,947,20.0,SHARED3FREE
10692278,195598317,596335,229795,24449789,eatout,1,False,1,Home,947,212,21.0,SHARED3PAY


### Run plan generation

In [110]:
from activitysim.abm.models.generate_beam_plans import get_trip_coords, generate_departure_times, sample_geoseries
from activitysim.abm.models.initialize_from_usim import get_zone_geoms_from_h3
import random

#### Create zones table

In [69]:
skims = pd.read_csv('https://beam-outputs.s3.amazonaws.com/output/detroit/detroit-200k__2020-05-29_22-09-51_mgd/ITERS/it.10/10.skimsOD.UrbanSim.Full.csv.gz')
h3_zone_ids = skims.origTaz.unique()

zone_geoms = get_zone_geoms_from_h3(h3_zone_ids)

zones = gpd.GeoDataFrame(h3_zone_ids, geometry=zone_geoms, crs="EPSG:4326")
zones.columns = ['h3_id', 'geometry']
zones['TAZ'] = list(range(1, len(h3_zone_ids) + 1))
zones = zones.set_index('TAZ')

block_geoms = gpd.read_file('../detroit/data/blocks.shp')
block_bounds = block_geoms.to_crs('EPSG:3078').unary_union

zones = zones.to_crs('EPSG:3078')
zones['geometry'] = zones['geometry'].intersection(block_bounds)
zones = zones.to_crs('EPSG:4326')

#### Generate plans

In [93]:
zones.geometry = zones.geometry.buffer(0)

In [94]:
zones.reset_index(inplace=True)

In [101]:
rand_point_zones = {}
for zone in zones[~zones['geometry'].is_empty].TAZ:
    size = 500
    polygon = zones[zones.TAZ == zone].geometry
    points = sample_geoseries(polygon, size, overestimate=2)
    rand_point_zones[zone] = points

In [282]:
trips = trips.set_index('trip_id').sort_values(['person_id', 'depart'])

In [283]:
df = trips[['origin']].reset_index().drop_duplicates('trip_id')

In [284]:
origins = []
for i, row in enumerate(df.itertuples(), 0):
    origins.append(random.choice(rand_point_zones[row.origin]))
origins = np.array(origins)

In [285]:
df['origin_x'] = origins[:, 0]
df['origin_y'] = origins[:, 1]

In [286]:
df = df.set_index('trip_id').reindex(trips.index)
trips['origin_x'] = df['origin_x']
trips['origin_y'] = df['origin_y']

In [287]:
trips['origin_purpose'] = trips.groupby(
    'person_id')['purpose'].shift(periods=1).fillna('Home')
trips['x'] = trips.origin_x
trips['y'] = trips.origin_y

In [288]:
trips[trips['person_id'] == 596338]

,person_id,household_id,tour_id,primary_purpose,trip_num,outbound,trip_count,purpose,destination,origin,depart,trip_mode,origin_x,origin_y,origin_purpose,x,y
trip_id,,,,,,,,,,,,,,,,,
195599177,596338,229795,24449897,work,1,True,1,work,156,947,6.0,WALK_LOC,-82.881102,42.581017,Home,-82.881102,42.581017
195598897,596338,229795,24449862,atwork,1,True,1,atwork,114,156,13.0,WALK,-82.913022,42.424501,work,-82.913022,42.424501
195598901,596338,229795,24449862,atwork,1,False,1,Work,156,114,16.0,WALK,-82.895691,42.440923,atwork,-82.895691,42.440923
195599181,596338,229795,24449897,work,1,False,1,Home,947,156,18.0,WALK_LOC,-82.911523,42.423217,Work,-82.911523,42.423217


In [292]:
df = trips[['person_id', 'depart']].reset_index().drop_duplicates('trip_id')
df['frac'] = np.random.rand(len(df),)

In [293]:
df.index.name = 'og_df_idx'

# PICK SHIT UP HERE ON MONDAY

In [ ]:
trips["tour_start"] = trips.tour_id.map(tours.start)
trips.sort_values(['person_id','tour_start','tour_id','inbound','trip_num'])

In [301]:
ordered = df.sort_values(by=['person_id', 'depart', 'frac']).reset_index()

In [302]:
ordered[ordered['person_id'] == 650]

,og_df_idx,trip_id,person_id,depart,frac
1959,1960,213513,650,7.0,0.243310
1960,1959,213233,650,7.0,0.358422
1961,1961,213237,650,9.0,0.189768
1962,1962,213238,650,9.0,0.717726
1963,1964,213518,650,19.0,0.727622
1964,1963,213517,650,19.0,0.969355


In [ ]:
df2 = df.sort_values(by=['person_id', 'depart']).reset_index()
df2['fractional'] = ordered.frac

In [236]:
trips['departure_time'] = generate_departure_times(trips)

In [305]:
trips[trips['person_id'] == 650]

,trip_id,person_id,household_id,tour_id,primary_purpose,trip_num,outbound,trip_count,purpose,destination,origin,depart,trip_mode
10611254,213233,650,187,26654,atwork,1,True,1,atwork,33,486,7.0,SHARED2FREE
10611255,213237,650,187,26654,atwork,1,False,2,othmaint,33,33,9.0,SHARED2FREE
10611256,213238,650,187,26654,atwork,2,False,2,Work,486,33,9.0,SHARED2PAY
10611257,213513,650,187,26689,work,1,True,1,work,486,821,7.0,DRIVEALONEPAY
10611258,213517,650,187,26689,work,1,False,2,shopping,486,486,19.0,DRIVEALONEPAY
10611259,213518,650,187,26689,work,2,False,2,Home,821,486,19.0,DRIVEALONEFREE


In [238]:
cols = [
    'person_id', 'departure_time', 'purpose', 'origin',
    'destination', 'trip_mode', 'x', 'y']
trips = trips[cols].sort_values(
    ['person_id', 'departure_time']).reset_index()

In [252]:
trips.groupby('person_id')['destination'].last()[(trips.groupby('person_id')['destination'].last() != trips.groupby('person_id')['origin'].first())]

person_id
650        821
1080       821
1583       821
2604       907
2656       907
          ... 
4391267    138
4392020    138
4392190    138
4392310    857
4392983    174
Name: destination, Length: 14188, dtype: int64

In [ ]:
trips.groupby(['person_id', 'depart'])

In [254]:
trips[trips['person_id'] == 4391267]

,trip_id,person_id,departure_time,purpose,origin,destination,trip_mode,x,y
14136932,1440335577,4391267,8.234,atwork,991,324,DRIVEALONEPAY,-83.184186,42.194760
14136933,1440335581,4391267,8.498,Work,324,991,DRIVEALONEFREE,-83.171514,42.231517
14136934,1440335889,4391267,8.892,work,138,991,DRIVEALONEFREE,-83.266094,42.110917
14136935,1440335865,4391267,18.121,social,138,150,DRIVEALONEFREE,-83.313741,42.115840
14136936,1440335893,4391267,18.564,Home,991,138,DRIVEALONEFREE,-83.181533,42.204011
14136937,1440335869,4391267,21.062,shopping,150,836,DRIVEALONEFREE,-83.058149,42.340641
14136938,1440335870,4391267,21.541,Home,836,138,DRIVEALONEPAY,-83.053665,42.332312


In [205]:
return_trip = pd.DataFrame(
    trips.groupby('person_id').agg({'x': 'first', 'y': 'first'}),
    index=trips.person_id.unique())

In [206]:
trips = trips.append(return_trip)
trips.reset_index(inplace=True)
trips.person_id.fillna(trips['index'], inplace=True)

In [207]:
trips[trips['person_id'] == 596338]

,index,trip_id,person_id,departure_time,purpose,origin,destination,trip_mode,x,y,PlanElementIndex,ActivityType
1950908,1950908,195599177.0,596338.0,6.720,work,947.0,156.0,WALK_LOC,-82.893245,42.576168,1.0,Home
1950909,1950909,195598897.0,596338.0,13.748,atwork,156.0,114.0,WALK,-82.910921,42.423763,3.0,work
1950910,1950910,195598901.0,596338.0,16.302,Work,114.0,156.0,WALK,-82.902152,42.444910,5.0,atwork
1950911,1950911,195599181.0,596338.0,18.027,Home,156.0,947.0,WALK_LOC,-82.918037,42.422579,7.0,Work
14667037,596338,NaN,596338.0,NaN,NaN,NaN,NaN,NaN,-82.893245,42.576168,NaN,NaN


In [208]:
trips['PlanElementIndex'] = trips.groupby('person_id').cumcount() * 2 + 1
trips = trips.sort_values(
    ['person_id', 'departure_time']).reset_index(drop=True)

In [209]:
trips[trips['person_id'] == 596338]

,index,trip_id,person_id,departure_time,purpose,origin,destination,trip_mode,x,y,PlanElementIndex,ActivityType
2473950,1950908,195599177.0,596338.0,6.720,work,947.0,156.0,WALK_LOC,-82.893245,42.576168,1,Home
2473951,1950909,195598897.0,596338.0,13.748,atwork,156.0,114.0,WALK,-82.910921,42.423763,3,work
2473952,1950910,195598901.0,596338.0,16.302,Work,114.0,156.0,WALK,-82.902152,42.444910,5,atwork
2473953,1950911,195599181.0,596338.0,18.027,Home,156.0,947.0,WALK_LOC,-82.918037,42.422579,7,Work
2473954,596338,NaN,596338.0,NaN,NaN,NaN,NaN,NaN,-82.893245,42.576168,9,NaN


In [210]:
trips['ActivityType'] = trips.groupby('person_id')['purpose'].shift(periods=1).fillna('Home')

In [216]:
trips[trips['person_id'] == 596338]

,index,trip_id,person_id,departure_time,purpose,origin,destination,trip_mode,x,y,PlanElementIndex,ActivityType,ActivityElement
2473950,1950908,195599177.0,596338.0,6.720,work,947.0,156.0,WALK_LOC,-82.893245,42.576168,1,Home,activity
2473951,1950909,195598897.0,596338.0,13.748,atwork,156.0,114.0,WALK,-82.910921,42.423763,3,work,activity
2473952,1950910,195598901.0,596338.0,16.302,Work,114.0,156.0,WALK,-82.902152,42.444910,5,atwork,activity
2473953,1950911,195599181.0,596338.0,18.027,Home,156.0,947.0,WALK_LOC,-82.918037,42.422579,7,Work,activity
2473954,596338,NaN,596338.0,NaN,NaN,NaN,NaN,NaN,-82.893245,42.576168,9,Home,activity


In [212]:
trips['ActivityElement'] = 'activity'

In [224]:
legs = trips[~pd.isnull(trips['trip_id'])]
legs['PlanElementIndex'] = legs['PlanElementIndex'] + 1
legs['ActivityElement'] = 'leg'
legs['ActivityType'] = None
trips['trip_id'] = None

In [225]:
legs[legs['person_id'] == 596338]

,index,trip_id,person_id,departure_time,purpose,origin,destination,trip_mode,x,y,PlanElementIndex,ActivityType,ActivityElement
2473950,1950908,195599177.0,596338.0,6.720,work,947.0,156.0,WALK_LOC,-82.893245,42.576168,2,Home,leg
2473951,1950909,195598897.0,596338.0,13.748,atwork,156.0,114.0,WALK,-82.910921,42.423763,4,work,leg
2473952,1950910,195598901.0,596338.0,16.302,Work,114.0,156.0,WALK,-82.902152,42.444910,6,atwork,leg
2473953,1950911,195599181.0,596338.0,18.027,Home,156.0,947.0,WALK_LOC,-82.918037,42.422579,8,Work,leg


In [226]:
trips = trips.append(legs).sort_values(['person_id', 'PlanElementIndex'])

In [227]:
trips[trips['person_id'] == 596338]

,index,trip_id,person_id,departure_time,purpose,origin,destination,trip_mode,x,y,PlanElementIndex,ActivityType,ActivityElement
2473950,1950908,195599177.0,596338.0,6.720,work,947.0,156.0,WALK_LOC,-82.893245,42.576168,1,Home,activity
2473950,1950908,195599177.0,596338.0,6.720,work,947.0,156.0,WALK_LOC,-82.893245,42.576168,2,Home,leg
2473951,1950909,195598897.0,596338.0,13.748,atwork,156.0,114.0,WALK,-82.910921,42.423763,3,work,activity
2473951,1950909,195598897.0,596338.0,13.748,atwork,156.0,114.0,WALK,-82.910921,42.423763,4,work,leg
2473952,1950910,195598901.0,596338.0,16.302,Work,114.0,156.0,WALK,-82.902152,42.444910,5,atwork,activity
2473952,1950910,195598901.0,596338.0,16.302,Work,114.0,156.0,WALK,-82.902152,42.444910,6,atwork,leg
2473953,1950911,195599181.0,596338.0,18.027,Home,156.0,947.0,WALK_LOC,-82.918037,42.422579,7,Work,activity
2473953,1950911,195599181.0,596338.0,18.027,Home,156.0,947.0,WALK_LOC,-82.918037,42.422579,8,Work,leg
2473954,596338,NaN,596338.0,NaN,NaN,NaN,NaN,NaN,-82.893245,42.576168,9,Home,activity


In [ ]:
trips[trips['Activity']]